# ECOSTRESS-EMIT Workshop Fall 2023 - Carpinteria Salt Marsh Analysis
Gregory Halverson
Jet Propulsion Laboratory, California Institute of Technology

Claire Villanueva-Weeks
Jet Propulsion Laboratory, California Institute of Technology

 This research was carried out at the Jet Propulsion Laboratory, California Institute of Technology, and was sponsored by ECOSTRESS and the National Aeronautics and Space Administration (80NM0018D0004). 

 © 2023. All rights reserved.
 
 ## Summary
 In this notebook we will open an EMIT L2A Reflectance product file in NETCDF4 format and an ECOSTRESS L2 Land Surface Temperature product file in GEOTIFF format. We will demonstrate using specific bands to calculate NDVI and use holoviews to plot the EMIT spectra and calculated NDVI. We will also demonstrate opening and mapping the ECOSTRESS L2 Land Surface Temperature product. 

> Note: This notebook has filepaths for data stored on the 2i2c Jupyterhub. If running locally you will need to change them manually.


## Importing Libraries

These are some built-in Python functions we need for this notebook, including functions for handling filenames and dates.

In [ ]:
# !mamba install -q -y cartopy earthaccess geoviews rasterstats folium geopandas hvplot holoviews matplotlib netCDF4 numpy pandas rasterio rasterstats rioxarray seaborn scikit-image shapely xarray

In [ ]:
import os, sys
from os.path import join, abspath, basename, splitext
from glob import glob
from datetime import datetime, date, timedelta
from zipfile import ZipFile
import warnings
warnings.filterwarnings("ignore")

We're using the `rioxarray` package for loading raster data from a GeoTIFF file, and we're importing it as `rxr`. We're using the `numpy` library to handle arrays, and we're importing it as `np`. We're using the `rasterio` package to subset the data.

In [ ]:
from osgeo import gdal
import rioxarray as rxr
import numpy as np
import rasterio as rio

We're using the `geopandas` library to load vector data from GeoJSON files, and we're importing it as `gpd`. We're using the `shapely` library to handle vector data and the `pyproj` library to handle projections.

In [ ]:
import geopandas as gpd
from shapely.geometry import Point, box
from shapely.ops import transform
from pyproj import Transformer

We're using the `pandas` library to handle tables, and we're importing it as `pd`.

In [ ]:
import pandas as pd

We're using the `seaborn` library to produce our graphs, and we're importing it as `sns`. We're using the `hvplot` library to produce our maps. 

In [ ]:
import seaborn as sns
import hvplot.xarray
import hvplot.pandas

In [ ]:
import earthaccess
import math
import netCDF4 as nc

Import the `emit_tools` module and call use from emit_tools import emit_xarray
help(emit_xarray) the help function to see how it can be used.

> Note: This function currently works with L1B Radiance and L2A Reflectance Data.

In [ ]:
sys.path.append('modules/')
from emit_tools import emit_xarray
help(emit_xarray)


## Defining Constants

These constants define the dimensions of our figures. Feel free to adjust these to fit your display.

In [ ]:
FIG_WIDTH_PX = 1080
FIG_HEIGHT_PX = 720
FIG_WIDTH_IN = 16
FIG_HEIGHT_IN = 9
FIG_ALPHA = 0.7
BASEMAP = "ESRI"
SEABORN_STYLE = "whitegrid"
sns.set_style(SEABORN_STYLE)

This is the location of the ECOSTRESS and EMIT product files

In [ ]:
DATA_DIRECTORY = "/home/jovyan/shared/ECOSTRESS-EMIT_data/data/" # FIXME set this to the common path in OpenScapes 
print(f"data directory: {DATA_DIRECTORY}")


## Loading and Mapping an ECOSTRESS LST granule

First, let's trying opening a data layer from a product file. 

In [ ]:
ECOSTRESS_fp = join(DATA_DIRECTORY, "ECOv002_L2T_LSTE_26921_001_11SKU_20230405T190258_0710_01_LST.tif")
ECOSTRESS_fp

We're using `rioxarray` to open the surface temperature product on the 11SKU tile covering the Carpinteria Salt Marsh. We're passing the filename of the GeoTIFF file directly into `rioxarray`.

In [ ]:
LST_K_raster = rxr.open_rasterio(ECOSTRESS_fp).squeeze('band', drop=True)
LST_K_raster

The `hvplot` package extends `xarray` to allow us to plot maps. We're reprojecting the raster geographic projection **EPSG 4326** to overlay on the basemap with a latitude and longitude graticule. We're using the `jet` color scheme to render temperature with a rainbow of colors with red meaning hot and blue meaning cool. We're setting the alpha to make the raster semi-transparent on top of the basemap. We're filtering out values lower than the 2% percentile and higher than the 98% percentile to make the variation in the image more visible.

The temperatures in the `L2T_LSTE` product are given in Kelvin. To convert them to Celsius, we subtract 273.15.

In [ ]:
LST_C_raster = LST_K_raster.copy()
LST_C_raster.data = LST_K_raster.data - 273.15

LST_C_map = LST_C_raster.rio.reproject("EPSG:4326").hvplot.image(
    geo=True,
    cmap="jet",
    tiles=BASEMAP,
    alpha=FIG_ALPHA,
    width=FIG_WIDTH_PX,
    height=FIG_HEIGHT_PX,
    clim=(LST_C_raster.quantile(0.02), LST_C_raster.quantile(0.98)),
    title=f"{splitext(basename(ECOSTRESS_fp))[0]} Surface Temperature (Celsius)"
)

LST_C_map = LST_C_map.options(xlabel="Longitude", ylabel="Latitude")
LST_C_map


## Loading an EMIT Reflectance Granule

So now that weve opened up and visualized an ECOSTRESS collection 2 LST granule, lets try opening a data layer from the EMIT product file.

EMIT L2A Reflectance Data are distributed in a non-orthocorrected spatially raw NetCDF4 (.nc) format. .nc files 

To open up the `.nc` file we will use the `netCDF4`, `xarray` and `emit_tools` libraries. 

In [ ]:
EMIT_fp = join(DATA_DIRECTORY, "EMIT_L2A_RFL_001_20230405T190323_2309513_003.nc")
EMIT_fp

Opening the file with nc allows us to see file information and the different groups, theres reflectance which were concerned with, sensor band parameters, and location. 

In [ ]:
ds_nc = nc.Dataset(EMIT_fp)
ds_nc

Now we will use the `emit_xarray` function from the `emit_tools` module wirh `ortho` set to `True` to orthorectify the L2A reflectance data and place it into an `xarray.Dataset`. 

>**For a detailed walkthrough of the orthorectification process using the GLT see section 2 of the How_to_Orthorectify.ipynb in the how-tos folder.**

In [ ]:
ds = emit_xarray(EMIT_fp,ortho=True)
ds

## Visualizing EMIT Spectral and Spatial Data

Here we picked out and mapped wavelengths nearest to 800 nm and 675 nm using the `.sel` function from `xarray` 

In [ ]:
ds.sel(wavelengths=800, method='nearest').hvplot.image(x='longitude', y='latitude',tiles=BASEMAP,cmap='viridis', aspect = 'equal', frame_width=400,title=f"{splitext(basename(EMIT_fp))[0]} ~800 nm") +\
    ds.sel(wavelengths=675, method='nearest').hvplot.image(x='longitude', y='latitude',tiles=BASEMAP,cmap='viridis', aspect = 'equal', frame_width=400,title=f"{splitext(basename(EMIT_fp))[0]} ~675 nm")

These are the `nearest` to 800 nm and 675 nm wavelengths, they can be used to calculate the NDVI using a ratio of the difference between between the wavelengths to the sum of the wavelengths.NDVI is a metric by which we can estimate vegetation greenness.

In [ ]:
NDVI = (ds.sel(wavelengths=800, method='nearest') - ds.sel(wavelengths=675, method='nearest'))/(ds.sel(wavelengths=800, method='nearest') + ds.sel(wavelengths=675, method='nearest'))
NDVI.hvplot.image(
    x='longitude', 
    y='latitude',
    width=FIG_WIDTH_PX,
    height=FIG_HEIGHT_PX,
    tiles=BASEMAP,
    cmap='RdYlGn', 
    aspect = 'equal', 
    title=f"{splitext(basename(EMIT_fp))[0]} Vegetation Index")


## Isolating Our Region of Interest 

Our ROI is the Carpinteria Salt Marsh Habitat, which is a marsh reserve here in Southern California that is home to many different species of plants and animals (more information- https://carpinteria.ucnrs.org/). Tomorrow when we take a fieldtrip there where there will be a guided tour where we will be learning more about the Carpinteria Salt Marsh and its ecology. 

To clip the raster image to the extent of the vector dataset, we want to subset the raster to the bounds of the vector dataset. This dataset is included here in GeoJSON format, which we'll load in as a geodatagrame using the `geopandas` package.

In [ ]:
landcoverfile = join(DATA_DIRECTORY,"landcover.geojson")
landcover_latlon = gpd.read_file(landcoverfile)

landcover_latlon

To align this vector dataset with the raster datasets, we need to project it to the UTM projection used for the ECOSTRESS rasters.

In [ ]:
LST_raster = LST_K_raster.copy()
landcover_UTM = landcover_latlon.to_crs(LST_raster.rio.crs)

landcover_UTM

This vector dataset contains polygons classifying the surface of the Carpinteria Salt Marsh into channel, salt flat, upland, pan, and marsh. You can see that this vector dataset contains 5 polygons that classify the Carpinteria Salt Marhs the Marsh into channel, salt flat, upland, pan, and marsh.

In [ ]:
landcover_colors = {
    "channel": "blue",
    "marsh": "yellow",
    "pan": "green",
    "salt flat": "white",
    "upland": "brown"
}

landcover_map = landcover_latlon.to_crs("EPSG:4326").hvplot.polygons(
    geo=True,
    color=landcover_UTM["type"].apply(lambda type: landcover_colors[type]),
    tiles=BASEMAP,
    alpha=FIG_ALPHA,
    width=FIG_WIDTH_PX,
    height=FIG_HEIGHT_PX,
    title="Carpinteria Salt Marsh Habitat Polygons"
)

landcover_map = landcover_map.options(xlabel="Longitude", ylabel="Latitude")
landcover_map

Now we can use the `clip` function from `rasterio` to mask out a subset of the the LST and NDVI datasets to the extent of the polygons from the vector dataset. Setting `all_touched` to `True` will include pixels that intersect with the edges of the polygons.

In [ ]:
LST_clip = LST_raster.rio.clip(landcover_latlon.geometry.values,landcover_latlon.crs, all_touched=True)
LST_map = LST_clip.hvplot.image(
    cmap='jet',
    alpha=FIG_ALPHA,
    width=FIG_WIDTH_PX,
    height=FIG_HEIGHT_PX,
    title = "Carpinteria Salt Marsh Surface Temperature (Celsius)"
) * landcover_UTM.hvplot(fill_color='none')

LST_map = LST_map.options(xlabel="Longitude", ylabel="Latitude")

LST_map

In [ ]:
NDVI_clip = NDVI.rio.clip(landcover_latlon.geometry.values,landcover_latlon.crs, all_touched=True)
NDVI_map = NDVI_clip.hvplot.image(
    cmap='RdYlGn',
    alpha=FIG_ALPHA,
    width=FIG_WIDTH_PX,
    height=FIG_HEIGHT_PX,
    title = "Carpinteria Salt Marsh Vegetation Index"
) * landcover_latlon.hvplot(fill_color='none')

NDVI_map = NDVI_map.options(xlabel="Longitude", ylabel="Latitude")

NDVI_map

Here's another way we can visualize them, we can map them side by side laid over a satellite basemap, also setting the `alpha` to a lower value to increase transparency of the raster datasets

In [ ]:
LSTmap1 = LST_clip.hvplot.image(
    tiles=BASEMAP,
    cmap='jet',
    alpha=.6,
    title = "Carpinteria Salt Marsh Surface Temperature (Celsius)"
)

NDVImap1 = NDVI_clip.hvplot.image(
    tiles=BASEMAP,
    cmap='RdYlGn',
    alpha=.6,
    title = "Carpinteria Salt Marsh Vegetation Index"
)

LSTmap1.options(xlabel="Longitude", ylabel="Latitude")+NDVImap1.options(xlabel="Longitude", ylabel="Latitude")